In [ ]:
import cv2
import numpy as np
import matplotlib.pyplot as plt
from google.colab.patches import cv2_imshow

path = '/content/drive/MyDrive/ABR/BEP/Day1/Lenna.png'

x = np.array(list(range(256)))

def img_info():
    img = cv2.imread(path, cv2.IMREAD_GRAYSCALE)
    print('shape :', img.shape)

def img_show(input):
    plt.imshow(cv2.cvtColor(input, cv2.COLOR_BGR2RGB))
    plt.show()

def plt_show(x, input):
    plt.bar(x, input)
    plt.show()

def color():
    img = cv2.imread(path, cv2.IMREAD_COLOR)
    plt.imshow(cv2.cvtColor(img, cv2.COLOR_BGR2RGB))
    plt.title('Title_color')
    plt.show()

def gray():
    img = cv2.imread(path, cv2.IMREAD_GRAYSCALE)
    plt.imshow(cv2.cvtColor(img, cv2.COLOR_BGR2RGB))
    plt.title('Title_gray')
    plt.show()

def Color_Extract():
    img = cv2.imread(path, cv2.IMREAD_COLOR)

    print('Green')
    green = np.zeros((220, 220, 3), dtype=np.uint8)
    for i in range(220):
        for j in range(220):
            green[i,j,1] = img[i,j,1]
    plt.imshow(cv2.cvtColor(green, cv2.COLOR_BGR2RGB))
    plt.show()

    print('Blue')
    blue = np.zeros((220, 220, 3), dtype=np.uint8)
    for i in range(220):
        for j in range(220):
            blue[i, j, 0] = img[i, j, 0]
    plt.imshow(cv2.cvtColor(blue, cv2.COLOR_BGR2RGB))
    plt.show()

    print('Red')
    red = np.zeros((220, 220, 3), dtype=np.uint8)
    for i in range(220):
        for j in range(220):
            red[i, j, 2] = img[i, j, 2]
    plt.imshow(cv2.cvtColor(red, cv2.COLOR_BGR2RGB))
    plt.show()

def histogram_PDF():
    img = cv2.imread(path, cv2.IMREAD_GRAYSCALE)
    PDF = np.zeros(256).astype(int)
    for i in range(220):
        for j in range(220):
            num = img[i][j]
            PDF[num] += 1
    #PDF = PDF/48400
    return PDF, img

def histogram_CDF():
    img = cv2.imread(path, cv2.IMREAD_GRAYSCALE)
    x = np.array(list(range(256)))
    PDF = np.zeros(256).astype(int)
    for i in range(220):
        for j in range(220):
            num = img[i][j]
            PDF[num] += 1
    #PDF = PDF/48400
    CDF = np.zeros(256)
    SUM = 0
    for i, j in enumerate(PDF.tolist()):
        SUM += j
        CDF[i] = SUM

    return CDF, img

def histogram_PDF_Sliding(offset):
    img = cv2.imread(path, cv2.IMREAD_GRAYSCALE)
    sliding = np.zeros((220, 220)).astype(int)
    for i in range(220):
        for j in range(220):
            sliding[i][j] = img[i][j] + offset
            if sliding[i][j] < 0 :
                sliding[i][j] = 0
            if sliding[i][j] > 255:
                sliding[i][j] = 255
    PDF = np.zeros(256).astype(int)
    for i in range(220):
        for j in range(220):
            num = sliding[i][j]
            PDF[num] += 1

    PDF = PDF/48400
    plt_show(x, PDF)
    plt.imshow(sliding, 'gray')

def histogram_PDF_Stretching():
    img = cv2.imread(path, cv2.IMREAD_GRAYSCALE)
    max = img.max()
    min = img.min()
    stretching = ((img - min) / (max - min)) * 255
    PDF = np.zeros(220*220)
    k = 0
    for i in range(220):
        for j in range(220):
            PDF[k] += stretching[i][j]
            k += 1

    plt.imshow(stretching, 'gray')
    plt.show()
    plt.hist(PDF, 255)
    plt.show()

def histogram_PDF_Shrink(s_min, s_max):
    img = cv2.imread(path, cv2.IMREAD_GRAYSCALE)
    max = img.max()
    min = img.min()
    shrink = np.zeros((img.shape[0], img.shape[1]))

    for i in range(img.shape[0]):
        for j in range(img.shape[1]):
            shrink[i][j] = ((img[i][j] - min)*(s_max - s_min)/(max - min)) + s_min

    cv2_imshow(shrink)
    plt.show()
    plt.hist(shrink.ravel(), 256, [0, 255])
    plt.show()

def histogram_CDF_Equlize():
    CDF, img = histogram_CDF()
    min = CDF.min()
    W, H = img.shape
    for i in range(256):
        CDF[i] = ((CDF[i] - min) / (W*H - min))*255
    Equlize = np.zeros(256)
    SUM = 0
    for i in range(256):
        SUM += CDF[i]
        Equlize[i] += SUM
    plt_show(x, Equlize)

def histogram_PDF_Quantization(rg):
    img = cv2.imread(path, cv2.IMREAD_GRAYSCALE)
    l = len(rg)

    for i in range(l-1):
        before = rg[i]
        next = rg[i+1]
        ran = (before <= img) & (img < next)
        img[ran] = before

    PDF = np.zeros(256)
    for i in range(220):
        for j in range(220):
            num = img[i][j]
            PDF[num] += 1
    PDF = PDF/48400

    plt_show(x, PDF)
    plt.imshow(img,'gray')
    plt.show()

def static_binarization(threshold):
    img = cv2.imread(path, cv2.IMREAD_GRAYSCALE)
    rg1 = img < threshold
    rg2 = img >= threshold

    img[rg1] = 0
    img[rg2] = 255
    plt.imshow(img, 'gray')
    plt.show()

def P_tile_binarization(T):
    CDF, img = histogram_CDF()
    CDF = CDF/48400
    prob = 1/T
    threshold = 0
    for i in range(256):
        if CDF[i] > prob:
           threshold = i
           break

    rg1 = img < threshold
    rg2 = img >= threshold

    img[rg1] = 0
    img[rg2] = 255
    plt.imshow(img, 'gray')
    plt.show()

def Iterative_binarization(T):
    PDF, img = histogram_PDF()
    m1 = PDF[:T].mean()
    m2 = PDF[T:].mean()
    T = int((m1 + m2)/2)
    print(T)
    before = 0
    plt_show(x, PDF)
    while 1:
        m1 = np.mean(PDF[:T])
        m2 = np.mean(PDF[T:])
        T = int((m1 + m2)/2)
        print(T)
        if abs(before - T) < 20 :
            break
        before = T
    T = int((before + T)/2)
    PDF = PDF/48400
    rg1 = img < T
    rg2 = img >= T

    img[rg1] = 0
    img[rg2] = 255
    plt.imshow(img, 'gray')
    plt.show()

def Otsu_binarization():
    T = list(range(1, 255))
    PDF, img = histogram_PDF()
    PDF = PDF/48400
    threshold = 0
    max = 0
    for i in T:
        w0 = PDF[:i].sum()
        w1 = 1 - w0
        m1 = PDF[:i].mean()
        m2 = PDF[i:].mean()
        sigb2 = w0*w1*((m1 - m2)**2)

        if sigb2 >= max:
            max = sigb2
            threshold = i

    rg1 = img < threshold
    rg2 = img >= threshold

    img[rg1] = 0
    img[rg2] = 255
    plt.imshow(img, 'gray')
    plt.show()

In [ ]:
histogram_PDF_Quantization([0, 150, 255])

In [ ]:
print('\n======= color ======')
color()
print('\n======= gray ======')
gray()
print('\n======= Color_Extract ======')
Color_Extract()
print('\n======= PDF ======')
PDF,_ = histogram_PDF();plt_show(x, PDF)
print('\n======= CDF ======')
CDF,_ = histogram_CDF();plt_show(x, CDF)
print('\n======= histogram_PDF_Sliding ======')
histogram_PDF_Sliding(10)
print('\n======= histogram_PDF_Stretching ======')
histogram_PDF_Stretching()
print('\n======= histogram_PDF_Shrink ======')
histogram_PDF_Shrink(150, 250)
print('\n======= histogram_CDF_Equlize ======')
histogram_CDF_Equlize()
print('\n======= histogram_PDF_Quantization ======')
histogram_PDF_Quantization([0, 50, 100, 150, 200, 255])
print('\n======= static_binarization ======')
static_binarization(100)
print('\n======= P_tile_binarization ======')
P_tile_binarization(2)
print('\n======= Iterative_binarization ======')
Iterative_binarization(30)
print('\n======= Otsu_binarization ======')
Otsu_binarization()